Data Visualization and Exploration
CPSC 5530
CRN 21428
Hunter Harris: zgt795
Project:

Import Libraries

In [65]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
import plotly.express as px

Load Data

In [66]:
fire_stations_df = pd.read_csv("Fire_Station_data.csv")
fire_incidents_df = pd.read_csv("Fire_Incidents_Detailed_Updated.csv")
budget_df = pd.read_csv("Chattanooga__Operating_Budget.csv")

/home/hunter/anaconda3/envs/data/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning:

Columns (2,18) have mixed types.Specify dtype option on import or set low_memory=False.



Set DataFrame Options

In [67]:
pd.set_option('display.max_columns', None)

Inspect Data Sets

In [68]:
# Print DataFrame information
print(fire_incidents_df.info())
print(fire_incidents_df['Specific Incident Code and Description'].value_counts().head(50))
print(fire_stations_df.info())
print(budget_df.info())

# Print DataFrames
print(fire_incidents_df.head(5))
print(fire_stations_df.head(5))
print(budget_df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661178 entries, 0 to 661177
Data columns (total 31 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   Incident Number                                  661178 non-null  object 
 1   Incident Date                                    661178 non-null  object 
 2   Shift Color                                      223014 non-null  object 
 3   Specific Incident Code and Description           655269 non-null  object 
 4   Generic Incident Code                            655269 non-null  float64
 5   Generic Incident Description                     655269 non-null  object 
 6   Incident Category                                651483 non-null  object 
 7   StreetNumber                                     661178 non-null  int64  
 8   StreetPrefix                                     120149 non-null  object 
 9   StreetName     

Clean budget DataFrame

In [69]:
# Filter out all departments except fire
budget_df = budget_df[budget_df['Department'] == 'Fire']

# Filter out all Programs except for Fire Stations
budget_df = budget_df[budget_df['Program'].str.contains("Fire Station")]
# budget_df = budget_df[budget_df['Fiscal Year'] == 2021]
budget_df = budget_df[budget_df['Fiscal Year'].isin([2015,2016,2017,2018])]

# Extract numerical value from Program and convert to numeric
budget_df['Program'] = pd.to_numeric(budget_df['Program'].str.extract('(\d+)', expand=False))
budget_df.rename(columns={'Program': 'Station'}, inplace=True)

# Drop unneeded columns
budget_df.drop(budget_df.columns[[1, 5, 7, 11]], axis=1, inplace=True)
budget_df = budget_df.reset_index(drop=True)

# Calculate budget by station
station_budget = pd.DataFrame(budget_df.groupby(['Station','Fiscal Year'])['Approved Amount'].sum())
station_budget.reset_index(inplace=True)

Clean Fire Station DataFrame

In [70]:
# Drop unneeded columns
fire_stations_df.drop(fire_stations_df.columns[[1, 2, 3, 9, 10]], axis=1, inplace=True)

# Extract station number and rename column
fire_stations_df['NAME'] = pd.to_numeric(fire_stations_df['NAME'].str.extract('(\d+)', expand=False))
fire_stations_df.rename(columns={'NAME': 'Station'}, inplace=True)

# Add missing phone numbers
fire_stations_df['PHONE_NO'].replace(np.NAN, "423-643-5600", inplace=True)

# Separate GPS coordinates into separate columns
fire_stations_df['the_geom'] = fire_stations_df['the_geom'].str.replace('POINT \(', ' ')
fire_stations_df['the_geom'] = fire_stations_df['the_geom'].str.replace(')', ' ')
fire_stations_df['the_geom'] = fire_stations_df['the_geom'].str.strip()
new = fire_stations_df["the_geom"].str.split(" ", n = 1, expand = True)
fire_stations_df["Lng"]= new[0]
fire_stations_df["Lat"]= new[1]

# Drop original GPS column
fire_stations_df.drop(fire_stations_df.columns[[0]], axis=1, inplace=True)

# Add budget column
fire_stations_df['Budget'] = fire_stations_df['Station'].map(station_budget['Approved Amount'])
print(fire_stations_df)

                   ADDRESS         CITY STATE  ZIP_CODE      PHONE_NO  \
0       4510 Bonny Oaks Dr  Chattanooga    TN     37416  423-892-2487   
1             3250 8th Ave  Chattanooga    TN     37407  423-867-4622   
2         809 S. Willow St  Chattanooga    TN     37404  423-622-2314   
3         6144 Dayton Blvd  Chattanooga    TN     37415  423-842-2253   
4           1033 Lupton Dr  Chattanooga    TN     37415  423-875-8420   
5             5 Francis St  Chattanooga    TN     37419  423-643-5600   
6         6418 Hixson Pike       Hixson    TN     37343  423-643-5600   
7           2181 Curtis St  Chattanooga    TN     37406  423-622-1414   
8            910 Wisdom St  Chattanooga    TN     37406  423-622-2042   
9        3003 Cummings Hwy  Chattanooga    TN     37419  423-825-0436   
10       5004 Brunswick Ln       Hixson    TN     37343  423-870-0638   
11   7700 East Brainerd Rd  Chattanooga    TN     37421  423-892-5300   
12  2103 Hickory Valley Rd  Chattanooga    TN     3

/tmp/ipykernel_6032/4139552511.py:12: FutureWarning:

The default value of regex will change from True to False in a future version.

/tmp/ipykernel_6032/4139552511.py:13: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



Clean Fire Incident DataFrame

In [71]:
# Drop Duplicates
fire_incidents_df.drop_duplicates(subset=['Incident Number'],
                       keep='first',
                       inplace=True)

acc_df = fire_incidents_df.reset_index(drop=True)

# Convert incident date to datatime object and create year column
fire_incidents_df['Incident Date'] = pd.to_datetime(fire_incidents_df['Incident Date'])
fire_incidents_df['Year'] = fire_incidents_df['Incident Date'].dt.year
fire_incidents_df['Month'] = fire_incidents_df['Incident Date'].dt.month

# Keep only years with budget data available
fire_incidents_df = fire_incidents_df[fire_incidents_df['Year'].isin(list(budget_df['Fiscal Year']))]

# Extract station number and rename column
fire_incidents_df['Responding Unit District Station'] = pd.to_numeric(fire_incidents_df['Responding Unit District Station'].str.extract('(\d+)', expand=False))
fire_incidents_df.rename(columns={'Responding Unit District Station': 'Station'}, inplace=True)
fire_incidents_df.dropna(how='all', subset=['Station'], inplace=True)
fire_incidents_df['Station'] = fire_incidents_df['Station'].astype(int)

#  Keep only records of known Fire Stations from fire_stations_df
fire_incidents_df = fire_incidents_df[fire_incidents_df['Station'].isin(list(fire_stations_df['Station']))]

# Add missing Data
col = ['Incident Category', 'Civilian Casualty', 'Firefighter Casualty', 'Property Loss', 'Property Value']
fire_incidents_df[col] = fire_incidents_df[col].replace(np.NAN, 0)

# Drop unneeded columns
fire_incidents_df.drop(fire_incidents_df.columns[[0, 2, 4, 7, 8, 9, 10, 11, 16, 18]], axis=1, inplace=True)

# Drop null values
fire_incidents_df = fire_incidents_df.dropna(subset=['Latitude','Longitude','Location'])
fire_incidents_df = fire_incidents_df.reset_index(drop=True)

print(fire_incidents_df)

            Incident Date             Specific Incident Code and Description  \
0     2017-03-24 16:30:00  321 EMS call  excluding vehicle accident with ...   
1     2017-03-24 17:00:00           151 Outside rubbish  trash or waste fire   
2     2017-03-24 18:01:00  251 Excessive heat  scorch burns with no ignition   
3     2017-03-24 20:30:00  321 EMS call  excluding vehicle accident with ...   
4     2017-03-24 20:44:00  321 EMS call  excluding vehicle accident with ...   
...                   ...                                                ...   
68479 2016-06-08 00:46:00  321 EMS call  excluding vehicle accident with ...   
68480 2016-06-08 01:12:00  745 Alarm system activation no fire unintentional   
68481 2016-06-08 06:05:00  321 EMS call  excluding vehicle accident with ...   
68482 2016-06-08 07:52:00        324 Motor vehicle accident with no injuries   
68483 2016-06-08 09:12:00       411 Gasoline or other flammable liquid spill   

                            Generic Inc

Build List for interactive heatmap

In [72]:
lat_long_list = []

# Create list for
for x in fire_incidents_df['Year'].sort_values().unique():
    lat_long_list.append(fire_incidents_df.loc[fire_incidents_df['Year'] == x,['Latitude','Longitude']].groupby(['Latitude','Longitude']).sum().reset_index().values.tolist())


Fire Station Visualizations

In [73]:
# Build Base Map
base_map = folium.Map(location=[35.043631, -85.309677], control_scale=True, zoom_start=11)

# Add labels for Fire Stations
for station in range(0, len(fire_stations_df)):
    html=f"""
        <h1> Station # {fire_stations_df.iloc[station]['Station']}</h1>
        <ul>
            <li>Address: {fire_stations_df.iloc[station]['ADDRESS']}</li>
            <li>City: {fire_stations_df.iloc[station]['CITY']}</li>
            <li>State: {fire_stations_df.iloc[station]['STATE']}</li>
            <li>Zip: {fire_stations_df.iloc[station]['ZIP_CODE']}</li>
            <li>Phone #: {fire_stations_df.iloc[station]['PHONE_NO']}</li>
            <li>Budget: $ {fire_stations_df.iloc[station]['Budget']}</li>
        </ul>
        """
    iframe = folium.IFrame(html=html, width=250, height=215)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(location=[fire_stations_df.iloc[station]['Lat'], fire_stations_df.iloc[station]['Lng']],
                  popup=popup,
                  icon=folium.DivIcon(html=f"""
                  <img src="https://i.imgur.com/zhm3d2b.png" width="40" height="40"/>
""")).add_to(base_map)

# Save Map for fire stations only
base_map.save(outfile= "Fire_Department_Only.html")

# Add Heatmap to basemap
HeatMapWithTime(lat_long_list, radius=10,index=[2015,2016,2017,2018], auto_play=True, max_opacity=0.3).add_to(base_map)

# Save Map for heatmap and fire station
base_map.save(outfile= "Fire_Department_Heatmap_2015_2021.html")

# Show map
base_map

another viz

In [74]:
fire_incidents_year_df = fire_incidents_df.groupby(['Year', 'Station']).size()
fire_incidents_year_df = fire_incidents_year_df.reset_index()
fire_incidents_year_df.rename(columns={0: 'Incidents'}, inplace=True)
print(fire_incidents_year_df)

fig = px.line(fire_incidents_year_df, x="Year", y="Incidents", color="Station", height=400)
fig.show()

    Year  Station  Incidents
0   2015        1       2583
1   2015        3        380
2   2015        4        917
3   2015        5       1375
4   2015        6        839
..   ...      ...        ...
72  2018       17        566
73  2018       19       1033
74  2018       20        929
75  2018       21       1037
76  2018       22        440

[77 rows x 3 columns]


Sunburst budget plot

In [75]:
fig = px.sunburst(budget_df, values='Approved Amount', path=['Station', 'Expense Category'], hover_name="Station", height=500 )
fig.write_html("Budget.html")
fig.show()

FireStation Budget over time

In [76]:
print(station_budget)
station_budget['Fiscal Year'] = pd.to_datetime(station_budget['Fiscal Year'], format="%Y")
fig = px.line(station_budget, x='Fiscal Year', y= 'Approved Amount', color='Station')
fig.update_layout(xaxis = dict(tickmode = 'array',tickvals = [2015, 2016, 2017, 2018],ticktext = [2015, 2016, 2017, 2018]))
fig.show()

    Station  Fiscal Year  Approved Amount
0         1         2015        4722866.0
1         1         2016        4521508.0
2         1         2017        4600581.0
3         1         2018        5123121.0
4         3         2015        1122470.0
..      ...          ...              ...
74       21         2018        1370470.0
75       22         2015        1179702.0
76       22         2016        1168481.0
77       22         2017        1239031.0
78       22         2018        1344229.0

[79 rows x 3 columns]


Another Viz

In [92]:
print(fire_incidents_df['Generic Incident Description'].value_counts())
fire_incidents_type_year_df = fire_incidents_df.groupby(['Year', 'Generic Incident Description']).size()
fire_incidents_type_year_df = fire_incidents_type_year_df.reset_index()
fire_incidents_type_year_df.rename(columns={0: 'Count'}, inplace=True)
print(fire_incidents_type_year_df)
fig = px.bar(fire_incidents_type_year_df, x='Generic Incident Description', y= 'Count', color='Generic Incident Description', animation_frame="Year", animation_group="Generic Incident Description", height=500)

fig.update_layout(margin=dict(l=20, r=20, t=20, b=200))
fig['layout']['updatemenus'][0]['pad']=dict(r= 10, t= 150)
fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 150,)

fig.write_html("Bar_Plot.html")



# import plotly.express as px
# df = px.data.gapminder()
# print(df)
#
# fig = px.bar(df, x="continent", y="pop", color="continent",
#   animation_frame="year", animation_group="country", range_y=[0,4000000000])


Rescue & Emergency Medical Service (EMS)               34000
False Alarm & False Call                               11493
Good Intent Call                                        8822
Service Call                                            7343
Hazardous Condition (no fire)                           3048
Fire                                                    2769
Overpressure Rupture, Explosion, Overheat (no fire)      342
Special                                                  292
Severe Weather & Natural Disaster                         43
Name: Generic Incident Description, dtype: int64
    Year                       Generic Incident Description  Count
0   2015                           False Alarm & False Call   2883
1   2015                                               Fire    724
2   2015                                   Good Intent Call   1915
3   2015                      Hazardous Condition (no fire)    776
4   2015  Overpressure Rupture, Explosion, Overheat (no ...     74
